In [1]:
import pandas as pd # Used for loading and handling data
import numpy as np # math functions
from collections import Counter 

# Splitting data
from sklearn.model_selection import train_test_split

# Comparing to existing implementation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Task 1.1 & 1.2 Implementing a decision tree learning algorithm from scratch

Below is the implemtation of task 1.1 and 1.2

In [2]:
class Node():
    def __init__(self, score=None, feature=None, threshold=None, left=None, right=None, 
                 prediction =None, pred_count = None):
        self.score = score
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.prediction = prediction
        self.pred_count = pred_count
    
    # For printing
    def __repr__(self):
        if self.prediction != None:
            return "PRED: "+str(self.prediction)+", count: "+str(self.pred_count)
        else:
            return ("N("+str(self.feature)+","+str(self.threshold)+")")
    
    # Comparing to other nodes
    def __eq__(self, other):

        if other == None:
            return False
        
        if other.left == None:
            return self.prediction == other.prediction and self.pred_count == other.pred_count

        else:
            return self.feature == (other.feature and self.threshold == other.threshold 
                                   and self.left.__eq__(other.left) and self.right.__eq__(other.right))
    
    # Flatten a 3 node tree to a single node
    def convert_to_pred(self, prediction, pred_count):
        self.score = None
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.prediction = prediction
        self.pred_count = pred_count

class DecisionTree():
    
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.root = None
        self.X_n = None
    
    def set_root(self, n):
        self.root = n
    
    def impurity(self, y, impurity_measure):
        _, class_counts = np.unique(y, return_counts=True)
        probs = class_counts / np.sum(class_counts)

        if impurity_measure == "entropy":
            impurity = -1 * np.sum(np.log2(probs) * probs)
        if impurity_measure == "gini":
            impurity = 1-np.sum(probs**2)

        return impurity

    
    # Finding optimal split of dataset based on impurity measure
    def opt_split(self, X, y, impurity_measure):
        m, n = X.shape

        c_score = self.impurity(y, impurity_measure)
        c_feature = None
        c_threshold = None
        X_left = None
        X_right = None
        y_left = None
        y_right =None

        for f in range(n):
            
            # One could use both mean and median to find split in dataset
            f_mean = np.mean(X[:,f])
            #f_median = np.median(X[:,f])

            left = np.where(X[:,f] < f_mean)
            right = np.where(X[:,f] >= f_mean)
            
            left_score = self.impurity(y[left], impurity_measure)
            right_score = self.impurity(y[right], impurity_measure)

            wheighted_score = (len(left[0])*left_score + (len(right[0])*right_score)) / (len(left[0]) + len(right[0]))

            if wheighted_score < c_score:
                c_score = wheighted_score
                c_feature = f
                c_threshold = f_mean
                X_left = X[left]
                X_right = X[right]
                y_left = y[left]
                y_right = y[right]
                

        return c_score, c_feature, c_threshold, X_left, X_right, y_left, y_right
    
    
    # Recursive function to fit a tree 
    def fit(self, X, y, impurity_measure, c_depth = 0):

        if c_depth == 0:
            m, n = X.shape
            self.X_n = n

        if c_depth == self.max_depth:
            p = Counter(y).most_common(1)[0]
            return Node(prediction=p[0], pred_count=p[1])

        if np.all(y == y[0]):
            p = Counter(y).most_common(1)[0]
            return Node(prediction=p[0], pred_count=p[1])

        if np.all(np.all(X == X[0,:], axis = 0)):
            p = Counter(y).most_common(1)[0]
            return Node(prediction=p[0], pred_count=p[1])

        else:
            (c_score, c_feature, c_threshold, X_left, X_right, y_left, y_right) = self.opt_split(X, y, impurity_measure)
            
            return Node(score = c_score,
                        feature = c_feature,
                        threshold = c_threshold,
                        left = self.fit(X_left, y_left, c_depth = c_depth+1, impurity_measure = impurity_measure),
                        right = self.fit(X_right, y_right, c_depth = c_depth+1, impurity_measure = impurity_measure)
                       )
    
    # Predicting on data using fitted tree 
    def predict(self, x, node=None):
        if node == None:
            if self.X_n != len(x):
                raise ValueError("Prediction input must be the same size as training data")
            node = self.root

        if node.right == None or node.left == None:
            return node.prediction
        if x[node.feature] < node.threshold:
            return self.predict(x, node.left)
        else:
            return self.predict(x, node.right)

# Recursive function to visualize tree in terminal
def print_tree(root, space = 0) :
    if (root == None) :
        return

    space += 4
    print_tree(root.right, space)
    for i in range(4, space):
        print(end = " ")
    print(root)
    print_tree(root.left, space)

In [3]:
def learn(X, y, impurity_measure= "entropy", max_depth = 5, prune = False):
    tree = DecisionTree(max_depth)
    if prune == True:
        
        # Pruning data
        X_t, X_prune, Y_t, Y_prune = train_test_split(X, y, test_size=0.3, random_state=42)
        root = tree.fit(X_t,Y_t, impurity_measure)
        tree.set_root(root)

        root_acc = accuracy_score(Y_prune,[tree.predict(x) for x in X_prune])

        # Prune function prunes all leaves. To prune entire tree it runs maximum depth-1
        pruned_root = prune_f(tree, root_acc, max_depth, X_prune, Y_prune)
        tree.set_root(pruned_root)
        
        for i in range(1,max_depth-1): # Cannot prune more than depth:
            pruned_root_i = prune_f(tree, root_acc, max_depth, X_prune, Y_prune)

            # Break if there there is not any more pruning changes
            if pruned_root_i == tree.root: 
                break

            tree.set_root(pruned_root_i)

        return tree
    else:
        root = tree.fit(X,y, impurity_measure)
        tree.set_root(root)
        return tree

def predict(x, tree):
    return tree.predict(x)


# Task 1.3 - Add reduced-error pruning

Helping function for pruning trees

In [4]:
def prune_f(tree, root_acc, depth, X_prune, Y_prune, node = None):

    #Base cases
    if node == None:
        node = tree.root
    if node.prediction != None:
        return node
        
    if (node.left.prediction != None) and (node.right.prediction != None):
        # If two leaves have identical label, combine
        if(node.left.prediction == node.right.prediction):
            return Node(prediction=node.left.prediction, pred_count = (node.left.pred_count+node.right.pred_count))
            
        else:
            # Replacing with majority label
            save_score, save_feature, save_threshold = node.score, node.feature, node.threshold
            l = node.left
            r = node.right

            left_majority = (l.pred_count > r.pred_count)
            
            node.left = None
            node.right = None

            if (left_majority):
                node.convert_to_pred(l.prediction, l.pred_count)
            else:
                node.convert_to_pred(r.prediction, r.pred_count)


            prune_acc = accuracy_score(Y_prune,[tree.predict(x) for x in X_prune])

            # Replacing node if accuracy is higher
            if (prune_acc > root_acc):
                if left_majority:
                    return Node(prediction=l.prediction,pred_count=l.pred_count)
                else:
                    return Node(prediction=r.prediction,pred_count=r.pred_count)
            # Else we keep the old
            else:
                return Node(save_score, save_feature, save_threshold, left = l, right = r)

    # Recursive call
    else:
        return Node(node.score, node.feature,node.threshold,
                    left = prune_f(tree, root_acc, depth, X_prune, Y_prune, node = node.left),
                    right = prune_f(tree, root_acc, depth, X_prune, Y_prune, node = node.right)
                    )

# Task 1.4 - Evaluate your algorithm

In [5]:
# Reading data
data = pd.read_csv("magic04.data", header=None)
data.columns = ["X"+str(i) for i in range(0,10)] + ["Y"]
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [6]:
# Splitting
X = data.drop(["Y"], axis = 1)
Y = data["Y"]

X_train, X_val_test, Y_train, Y_val_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=42)

In [7]:
#depth = list(range(2, 15))
depth = list(range(2,15)) # For grading you should probably run less samples to save time
impurity_measure = ["entropy", "gini"]
prune = [True, False]

df_l = []

# Checking accuracy for all combinatios of settings on validation data
for d in depth:
    for i in impurity_measure:
        for p in prune:
            clf = learn(X_train.to_numpy(),Y_train.to_numpy(), max_depth = d, impurity_measure=i, prune=p)
            val_pred = [predict(x, clf) for x in X_val.to_numpy()]
            acc = accuracy_score(Y_val, val_pred)*100
            df_l.append([d,i,p,acc])
            print("Descicion tree with max depth: "+str(d) + ", impurity measure: "+i+", prune:",str(p)+
                  ", Accuracy: {:3.2f}%".format(acc))


Descicion tree with max depth: 2, impurity measure: entropy, prune: True, Accuracy: 75.18%
Descicion tree with max depth: 2, impurity measure: entropy, prune: False, Accuracy: 75.18%
Descicion tree with max depth: 2, impurity measure: gini, prune: True, Accuracy: 75.18%
Descicion tree with max depth: 2, impurity measure: gini, prune: False, Accuracy: 75.18%
Descicion tree with max depth: 3, impurity measure: entropy, prune: True, Accuracy: 78.97%
Descicion tree with max depth: 3, impurity measure: entropy, prune: False, Accuracy: 78.90%
Descicion tree with max depth: 3, impurity measure: gini, prune: True, Accuracy: 78.97%
Descicion tree with max depth: 3, impurity measure: gini, prune: False, Accuracy: 78.90%
Descicion tree with max depth: 4, impurity measure: entropy, prune: True, Accuracy: 80.83%
Descicion tree with max depth: 4, impurity measure: entropy, prune: False, Accuracy: 80.72%
Descicion tree with max depth: 4, impurity measure: gini, prune: True, Accuracy: 80.83%
Descicion

In [13]:
# I will comment this out for grading in case grader dont have plotly installed

#import plotly.express as px #For visualization

#Plotting data
#df = pd.DataFrame(df_l, columns = ['Depth', 'Impurity', 'Prune', 'Accuracy'])
#px.line(df, x="Depth", y = 'Accuracy', color='Impurity', symbol='Prune', title="Model accuracy on validation data")

![Accuracy](acc.png)

**Which setting should you select for this data (entropy or
gini, pruning or no pruning)?** 
- The settings produce fairly similar perfomance, though the gini impurity measures slightly outperforms entropy.
- We also see that pruning decreases accuracy for depth <10, but increases for larger depths
- We select gini index with depth of 8 based on validation data.

**What is your estimate for the performance the selected model on unseen data points? Report how you arrived at the
conclusions.**
- We estimate based on the visualization that the model will predict correctly in around 85& of cases on unseen data.

In [9]:
# Test accuracy using best parameters as stated above

clf = learn(X_train.to_numpy(),Y_train.to_numpy(), max_depth = 9, impurity_measure="gini", prune=False)
clf_Y_test_pred = [predict(x, clf) for x in X_test.to_numpy()]
clf_acc = accuracy_score(Y_test, clf_Y_test_pred)*100
print("DecisionTree Test accuracy: {:3.2f}%".format(clf_acc))

DecisionTree Test accuracy: 85.21%


Our estimation is confirmed by test data.

# Task 1.5 - Compare to an existing implementation


In [10]:
tree_param = {
    'criterion':['gini','entropy'],
    'max_depth':[3,4,5,6,7,8,9,10],
    'min_samples_split': [2,3,4],
    'min_samples_leaf': [1,2]
    }
sklearn_clf = GridSearchCV(DecisionTreeClassifier(random_state=42), tree_param, cv=5, verbose=1)
sklearn_clf.fit(X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

![sklearn tree](tree.png)

Visualization of the tree produced by sklearn

In [11]:
print("sklearn DecisionTreeClassifier best params: " + str(sklearn_clf.best_params_))
sklearn_clf_Y_test_pred = sklearn_clf.predict(X_test)
print("sklearn DecisionTreeClassifier test accuracy: {:3.2f}%".format(accuracy_score(Y_test, sklearn_clf_Y_test_pred)*100))
print("Versus implementation form scratch test accuracy: {:3.2f}%".format(clf_acc))

sklearn DecisionTreeClassifier best params: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 2}
sklearn DecisionTreeClassifier test accuracy: 85.63%
Versus implementation form scratch test accuracy: 85.21%


**How does your implementation fare against this implementation in terms of
accuracy and speed? Can you explain the (possible) differences?**
- Our implementation is strikingly similar in performance and optimal settings. As we saw in the graph above there was not a significant difference between gini and entropy, and the optimal depth range was 8-9.
- The accuracy is similar: 85,63% VS 85,21%
- The speed is also similar for fitting
